# Project 1: Navigation

Author: Vignesh.b.Yaadav

In this project, I have built a value-based method, Deep-Q-Learning  Algorithm and its varients to learn an optimal policy in a model-free re reinforcement learning using a Unity environment.  which consists of a continuous state space of 37 dimensions, intending to navigate around and collect yellow bananas (reward: +1) while avoiding blue bananas (reward: -1). There are 4 actions to choose from: move left, move right, move forward, and move backward. An agent choosing action randomly 

# Implementation

At the core of the learning, the algorithm is  Deep Q-learning, which surpassed human-level performance in Atari games. It is an off-policy approximation learning algorithm where the policy being approximated is different from the policy being learned. To understand the basics idea of Q-learning is to learn the action-value function often denoted as Q(s, a), where (s)represents the current state and (a) represents the action. Q-learning is a derivate of Temporal-Difference learning where unlike Monte-Carlo methods, we can learn from each step rather than waiting for an episode to complete. The basic idea is that once we take action and update the experience into a new state, we use the current Q-value of that state as the estimate for future rewards.
There's one niche problem here. Since our space is continuous, we can't use a tabular/scalar representation. Hence, we use Function Approximation. The core idea behind a function approximator is to introduce a new parameter that helps us to obtain an approximation of the Q(s, a). So, this becomes a supervised-learning problem where the approximate represents the expected value and becomes the target. We use mean-square error as  (loss function) and update the weights accordingly using gradient-descent. Now all left with us is the function approximator. That's Deep Learning right! We use a neural network as a function approximator here. More specifically, we choose a 2-hidden layer network with both the layers having 64 hidden units with relu activation applied after each fully-connected layer. Adam was used as the optimizer for finding the optimal weights.

# About Experience Replay

The idea here is, If we keep learning from experiences as they come, then we have basically observed a sequence of observations each of which is linked to each other. We iterate for a few episodes and store each of the experiences in the buffer. After a fixed number of iterations, we sample a few experiences from this replay buffer and use that to calculate the loss and eventually update the parameters. Sampling randomly this way breaks the sequential nature of experiences and stabilizes learning. It also helps us use experience more than once.